In [24]:
from gensim.models.word2vec import Word2Vec, Vocab
from gensim import utils, matutils
import numpy as np
from sklearn.cluster import KMeans
from sklearn.preprocessing import normalize
from collections import defaultdict
import scipy

In [5]:
model = Word2Vec.load_word2vec_format("glove.6B/glove.6B.300d_2.txt", binary=False)

In [6]:
word2index = dict()
for index, word in enumerate(model.index2word):
    word2index[word] = index 

In [7]:
stop_set = set()
for line in open("stopwords.txt"):
    stop_set.add(line.strip())

In [11]:
infile = "SCWS/ratings.txt"

In [42]:
total_sim = 0
sims = defaultdict(list)
vocab = set([])
scores_list = []
sim_list = []

for line_num, line in enumerate(open(infile)):
    tokens = line.strip().split("\t")
    w1, pos1, w2, pos2 = tokens[1: 5]
    sent1, sent2 = tokens[5: 7]
    score = float(tokens[7])

    """
    if w1 in model.vocab and w2 in model.vocab:
        sims[w1].append((w2, score))
        sims[w2].append((w1, score))
        vocab.add(w1)
        vocab.add(w2)

    """
    if w1 in model.vocab and w2 in model.vocab:
        v1 = model.syn0[word2index[w1.lower()]]
        v2 = model.syn0[word2index[w2.lower()]]
        vsim = np.dot(v1.T, v2)
        total_sim += 1
        scores_list.append(score)
        sim_list.append(vsim)

In [43]:
print scipy.stats.spearmanr(scores_list, sim_list)[0]  

0.550334325704


In [37]:
model.syn0norm = normalize(model.syn0, axis=1)

In [38]:
print("Evaluation vocabulary size: %s" % len(vocab))
# Evaluate the matrix by creating a vector of all_scores for data
# and all_dists for mat's distances. 
all_scores = []
all_dists = []
for word in vocab:
    vec = model.syn0norm[word2index[word.lower()]]
    vals = sims[word]
    cmps, scores = zip(*vals)
    all_scores += scores
    all_dists += [np.dot(vec.T, model.syn0norm[word2index[w.lower()]]) for w in cmps]

# Return just the rank correlation coefficient (index [1] would be the p-value):
print scipy.stats.spearmanr(all_scores, all_dists)[0]  

Evaluation vocabulary size: 1682
0.561327419139


In [35]:
len(all_scores)

3954

In [41]:
all_dists

[0.30053815,
 0.030257169,
 0.24736096,
 0.069801666,
 0.33145073,
 0.15840012,
 0.18129177,
 -0.030887797,
 1.0,
 1.0,
 0.42560375,
 0.069410264,
 0.14237478,
 0.24719837,
 0.11101727,
 -0.057083964,
 0.14397699,
 0.28685749,
 0.16850507,
 0.06987229,
 0.30766457,
 0.404073,
 0.40256989,
 0.35670543,
 0.0037482567,
 1.0,
 1.0,
 0.17860121,
 0.31602925,
 0.33331004,
 0.99999994,
 0.99999994,
 0.31031305,
 -0.072197206,
 0.181394,
 0.4581731,
 0.20071185,
 0.03809239,
 0.10069267,
 0.31612718,
 0.048016101,
 0.053410809,
 0.40506244,
 0.21272033,
 0.1963769,
 0.37438771,
 0.035221815,
 0.25804901,
 0.25851688,
 0.14713573,
 0.31918162,
 -0.1123874,
 0.48785377,
 0.48785377,
 0.2057907,
 0.23537156,
 -0.060163323,
 0.99999988,
 0.99999988,
 0.63091612,
 0.22077571,
 0.63091612,
 0.34746376,
 0.22267601,
 0.33617967,
 0.3562718,
 0.057166256,
 0.29704618,
 0.14397699,
 0.053497657,
 0.14594695,
 0.076849945,
 0.058886047,
 0.542436,
 0.542436,
 0.17436127,
 0.18489967,
 0.25684029,
 0.030

In [40]:
total_sim

0

In [11]:
tokens

['2003',
 'Arafat',
 'n',
 'terror',
 'n',
 'Yasser <b> Arafat </b> or by his kunya Abu Ammar , was a Palestinian leader and a Laureate of the Nobel Prize . He was Chairman of the Palestine Liberation Organization (PLO) , President of the Palestinian National Authority (PNA) , and leader of the Fatah political party , which he founded in 1959 .',
 'David Forte states that the primary difference between <b> terror </b> and terrorism is that while terror can be neutrally evil , i.e. random violence committed by robbers , rapists and even soldiers , terrorism has the additional political or moral dimension , being the systemised use of randomly focused violence by organised groups against civilian targets to effect a political objective .',
 '2.72',
 '0.0',
 '0.0',
 '5.0',
 '1.0',
 '0.0',
 '9.0',
 '3.2',
 '0.0',
 '9.0',
 '0.0']

In [19]:
vsim

2.2622585

In [15]:
print sent1, "\n", sent2

Yasser <b> Arafat </b> or by his kunya Abu Ammar , was a Palestinian leader and a Laureate of the Nobel Prize . He was Chairman of the Palestine Liberation Organization (PLO) , President of the Palestinian National Authority (PNA) , and leader of the Fatah political party , which he founded in 1959 . 
David Forte states that the primary difference between <b> terror </b> and terrorism is that while terror can be neutrally evil , i.e. random violence committed by robbers , rapists and even soldiers , terrorism has the additional political or moral dimension , being the systemised use of randomly focused violence by organised groups against civilian targets to effect a political objective .


In [20]:
print avg_sim

2.72
